In [1]:
import pandas as pd


data = pd.read_csv(r'F:\Gdelt-ergm\18_19ERGMedges.csv')

# Pivot the data
parent_distance = data.pivot_table(values='p', index='node1', columns='node2')

# Fill NaN values with 0
parent_distance.fillna(0, inplace=True)

parent_distance.to_csv(r'F:\Gdelt-ergm\parent_relationship.csv', index=False)


In [22]:
import pandas as pd

# Load the data
data = pd.read_csv(r'F:\Gdelt-ergm\18_19ERGMFINAL.csv')

dist_cepii = pd.read_excel(r'F:\Gdelt-ergm\dist_cepii.xls')

# Combine Source and Target into a single node column
nodes = pd.concat([data["Source"], data["Target"]]).unique()

# Create a DataFrame from the unique nodes
nodes_df = pd.DataFrame(nodes, columns=["node"])

# Create a dictionary to map node names to their corresponding codes
code_mapping = dict()
for index, row in data.iterrows():
    code_mapping[row["Source"]] = row["SourceCode"]
    code_mapping[row["Target"]] = row["TargetCode"]

# Add the 'code' column to the nodes DataFrame using the map() function
nodes_df["code"] = nodes_df["node"].map(code_mapping)

# Perform a self-join (Cartesian product) on the nodes_df DataFrame
node_pairs_df = nodes_df.merge(nodes_df, how='cross')

# Rename the columns to 'node1', 'code1', 'node2', 'code2'
node_pairs_df.columns = ['node1', 'code1', 'node2', 'code2']

# Merge node_pairs_df with dist_cepii to add the 'dist' column
node_pairs_with_dist_df = node_pairs_df.merge(
    dist_cepii, left_on=['code1', 'code2'], right_on=['iso_o', 'iso_d'], how='left'
)

# Drop the unnecessary 'iso_o' and 'iso_d' columns
node_pairs_with_dist_df.drop(['code1','code2','iso_o', 'iso_d','comlang_off','comlang_ethno','colony','comcol','curcol','col45','smctry','distcap','distw','distwces'], axis=1, inplace=True)

# Print the result
print(node_pairs_with_dist_df)
node_pairs_with_dist_df.to_csv(r'F:\Gdelt-ergm\node_pairs_with_dist_df.csv')

        node1                               node2  contig        dist
0      UNICEF                              UNICEF       0  1161.07400
1      UNICEF                WORLD ECONOMIC FORUM       0  6272.28500
2      UNICEF                             ABB LTD       0  6272.28500
3      UNICEF           WORLD HEALTH ORGANIZATION       0  6272.28500
4      UNICEF               ACTION AGAINST HUNGER       0  1161.07400
...       ...                                 ...     ...         ...
89995    YWCA                            INTERSOS       0   748.71170
89996    YWCA                      CITRIX SYSTEMS       0  6272.28500
89997    YWCA                     HUTCHISON ESSAR       0  6249.01300
89998    YWCA  BANK FOR INTERNATIONAL SETTLEMENTS       0    76.42681
89999    YWCA                                YWCA       0    76.42681

[90000 rows x 4 columns]


In [24]:
import numpy as np

# Create a mapping of node names to integer indices
node_index_mapping = {node: i for i, node in enumerate(nodes_df['node'])}

# Add 'index1' and 'index2' columns to the DataFrame using the mapping
node_pairs_with_dist_df['index1'] = node_pairs_with_dist_df['node1'].map(node_index_mapping)
node_pairs_with_dist_df['index2'] = node_pairs_with_dist_df['node2'].map(node_index_mapping)

# Fill missing values in the 'contig' column with 0
node_pairs_with_dist_df['contig'] = node_pairs_with_dist_df['contig'].fillna(0)

# Fill missing values in the 'dist' column with the maximum value plus one
max_dist = node_pairs_with_dist_df['dist'].max()
node_pairs_with_dist_df['dist'] = node_pairs_with_dist_df['dist'].fillna(max_dist + 1)

# Pivot the DataFrame to create the 300x300 matrices for 'contig' and 'dist'
contig_matrix = node_pairs_with_dist_df.pivot(index='index1', columns='index2', values='contig').values
dist_matrix = node_pairs_with_dist_df.pivot(index='index1', columns='index2', values='dist').values

# Convert the matrices to NumPy arrays of shape (300, 300)
contig_matrix = np.array(contig_matrix)
dist_matrix = np.array(dist_matrix)

# Print the matrices
print("Contig matrix:")
print(contig_matrix)

print("\nDist matrix:")
print(dist_matrix)

Contig matrix:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Dist matrix:
[[ 1161.074    6272.285    6272.285   ... 11761.81     6272.285
   6272.285  ]
 [ 6272.285      76.42681    76.42681 ...  6249.013      76.42681
     76.42681]
 [ 6272.285      76.42681    76.42681 ...  6249.013      76.42681
     76.42681]
 ...
 [11761.81     6249.013    6249.013   ...   681.9816   6249.013
   6249.013  ]
 [ 6272.285      76.42681    76.42681 ...  6249.013      76.42681
     76.42681]
 [ 6272.285      76.42681    76.42681 ...  6249.013      76.42681
     76.42681]]


In [25]:
# Save the matrices to CSV files
np.savetxt(r'F:\Gdelt-ergm\contig_matrix.csv', contig_matrix, delimiter=',')
np.savetxt(r'F:\Gdelt-ergm\dist_matrix.csv', dist_matrix, delimiter=',')

In [26]:
# Check for missing data in node_pairs_with_dist_df
missing_data = node_pairs_with_dist_df[node_pairs_with_dist_df.isnull().any(axis=1)]
print("Missing data:")
print(missing_data)

Missing data:
Empty DataFrame
Columns: [node1, node2, contig, dist, index1, index2]
Index: []
